In [2]:
import os
import json
import re
import mmap
import pickle
from typing import List, Tuple, Dict
from functools import cache
from pathlib import Path
from dataclasses import dataclass, field
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from loguru import logger
from diskcache import Cache

from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage, Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

from bert_score import score
import torch
import multiprocessing


---
# Подгружаю модель

In [1]:
from main import Config, RAGSystem

config = Config()
rag_system = RAGSystem(config)

2024-11-10 10:30:20.417 | INFO     | main:get_optimal_threads:64 - Detected 20 CPU cores
2024-11-10 10:30:20.417 | INFO     | main:__post_init__:86 - Using device: cpu
2024-11-10 10:30:20.418 | INFO     | main:__post_init__:87 - Using 20 threads
2024-11-10 10:30:20.418 | INFO     | main:initialize:205 - Initializing ModelManager
2024-11-10 10:30:25.318 | INFO     | main:initialize:216 - Loading existing database
2024-11-10 10:30:25.336 | INFO     | main:initialize_retrievers:223 - Initializing retrievers
2024-11-10 10:30:25.342 | INFO     | main:initialize_retrievers:236 - Successfully initialized all retrievers


---
# Подгружаю тестовые датасет

In [4]:
import pandas as pd

PATH = './data/test_data_mediawise/'

sample_submission = pd.read_csv(PATH + 'sample_submission.csv')
print(sample_submission.shape)
sample_submission.head()

(188, 4)


,question,filename,slide_number,answer
0,Чем занимается платформа MEDIADESK?,0,0,Технологическая платформа автоматизированной з...
1,Чем отличаются возможности слабого (NARROW AI)...,0,0,Слабый (NARROW AI) - получение информации и пе...
2,Сколько процентов рутинных процессов отдаются ...,0,0,0.8
3,Как усложняться механики кампаний при использо...,0,0,Появление новых возможностей для персонификаци...
4,На кого чаще всего подписаны пользователи соци...,49,0,"FRIENDS, FAMILY, OR OTHER PEOPLE YOU KNOW"


In [6]:
test = pd.read_csv(PATH + 'test.csv')
print(test.shape)
test.head()

(188, 1)


,question
0,Чем занимается платформа MEDIADESK?
1,Чем отличаются возможности слабого (NARROW AI)...
2,Сколько процентов рутинных процессов отдаются ...
3,Как усложняться механики кампаний при использо...
4,На кого чаще всего подписаны пользователи соци...


---
# ответы модели

In [ ]:
# Get context and generate response using RAG system
context, docs, confidence, metadata = rag_system.get_relevant_content(user_input)
response = rag_system.generate_response(user_input, context)